In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
import os
from functools import reduce
import warnings 
warnings.filterwarnings('ignore')

In [2]:
# 분류모델 import 
# Logistic Regression,SGD,KNN,SVM,Decision Tree, Random Forest, LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report,roc_auc_score,f1_score

lr = LogisticRegression()
knn = KNeighborsClassifier()
svm = SVC(probability=True)
dtc = DecisionTreeClassifier()
rfc = RandomForestClassifier()
lgbm = LGBMClassifier()

In [3]:
df_coldwave = pd.read_csv('./data/preprocessed/df_coldwave.csv',encoding='cp949',index_col=0)
df_near = pd.read_csv('./data/preprocessed/preprocessed_주변국 기상데이터_199601to202108.csv',index_col=0)

In [4]:
df_coldwave.drop(['연도','날짜','지점'],axis=1,inplace=True)

In [5]:
# df_coldwave.reset_index(inplace=True)

In [6]:
# 중복된 날짜 제거 
df_coldwave = df_coldwave.drop_duplicates()

In [7]:
df_coldwave

,일시,재난일어난날
0,1973-01-03,1973-01-03
16,1973-01-04,1973-01-04
18,1973-01-06,1973-01-06
19,1973-01-11,1973-01-11
22,1973-01-12,1973-01-12
...,...,...
21950,2022-02-20,2022-02-20
21952,2022-02-21,2022-02-21
21960,2022-02-22,2022-02-22
21963,2022-02-23,2022-02-23


In [8]:
df_near

,도시명,일시,평균기온,최고기온,최저기온,습도,풍향,풍속,강수량,해면기압,이슬점 온도,군집
0,청진,1996-01-01,-5.528571,-0.1,-9.9,53.571429,330.0,0.571429,0.0,1018.685714,-13.657143,n2
1,청진,1996-01-02,-3.825000,1.3,-8.2,72.875000,80.0,0.500000,0.0,1014.037500,-8.025000,n2
2,청진,1996-01-03,-6.912500,-1.4,-11.7,48.375000,330.0,3.000000,0.0,1010.537500,-16.100000,n2
3,청진,1996-01-04,-9.475000,-3.4,-19.2,56.500000,320.0,0.250000,0.0,1020.800000,-16.625000,n2
4,청진,1996-01-05,-4.487500,0.5,-9.9,71.750000,290.0,0.375000,0.0,1021.800000,-8.837500,n2
...,...,...,...,...,...,...,...,...,...,...,...,...
9370,후쿠오카,2021-08-27,29.175000,32.8,25.6,70.495833,30.0,5.125000,0.0,1014.391667,23.162500,n5
9371,후쿠오카,2021-08-28,28.641667,32.3,25.1,72.362500,40.0,4.958333,0.0,1014.325000,23.000000,n5
9372,후쿠오카,2021-08-29,28.908333,32.4,25.5,72.687500,290.0,5.041667,0.0,1014.754167,23.375000,n5
9373,후쿠오카,2021-08-30,28.904167,32.5,25.8,75.795833,310.0,4.541667,0.0,1015.587500,24.112500,n5


In [9]:
# 주변국 데이터 전처리

In [10]:
df_near = df_near.groupby(['군집','일시']).mean()

In [11]:
df_near.reset_index(inplace=True)

In [12]:
df_near

,군집,일시,평균기온,최고기온,최저기온,습도,풍향,풍속,강수량,해면기압,이슬점 온도
0,n1,1996-01-01,-3.639583,0.500000,-8.087500,48.419643,218.750000,1.169643,0.000000,1022.948810,-13.566369
1,n1,1996-01-02,0.900000,3.750000,-2.812500,71.203125,165.000000,2.515625,0.446631,1017.043750,-3.859375
2,n1,1996-01-03,-6.095313,0.075000,-8.775000,52.091518,253.750000,4.433265,0.446631,1021.464062,-14.331250
3,n1,1996-01-04,-7.325000,-1.975000,-12.437500,59.312500,198.348983,1.484375,0.445600,1025.557812,-14.362500
4,n1,1996-01-05,-1.073214,1.175000,-3.050000,80.558036,230.524917,1.966518,1.623677,1021.357143,-4.158482
...,...,...,...,...,...,...,...,...,...,...,...
65620,n7,2021-08-27,25.229167,28.333333,22.500000,78.830556,240.000000,2.125000,-0.047822,1012.870833,21.366667
65621,n7,2021-08-28,26.490476,30.100000,23.133333,81.301190,106.666667,1.404762,-0.739609,1012.662500,22.993452
65622,n7,2021-08-29,25.787500,29.033333,23.233333,87.795833,210.000000,2.083333,-0.702848,1013.058333,23.450000
65623,n7,2021-08-30,27.412500,30.700000,25.233333,83.704167,220.000000,1.916667,-0.625120,1012.554167,24.179167


In [14]:
tdf = pd.DataFrame({'일시':df_near['일시'].unique()})
tdf

,일시
0,1996-01-01
1,1996-01-02
2,1996-01-03
3,1996-01-04
4,1996-01-05
...,...
9370,2021-08-27
9371,2021-08-28
9372,2021-08-29
9373,2021-08-30


In [15]:
gunjip = list(df_near['군집'].unique())
for hpa in gunjip:
    tmp = df_near[df_near['군집']==hpa].drop('군집',axis=1)
    tmp.columns = ['일시', f'{hpa}일시', f'{hpa}평균기온', f'{hpa}최고기온', f'{hpa}습도', f'{hpa}풍향', f'{hpa}풍속', f'{hpa}강수량', f'{hpa}해면기압', f'{hpa}이슬점 온도']
    tdf = pd.merge(tdf,tmp, how='left',on='일시')
tdf

,일시,n1일시,n1평균기온,n1최고기온,n1습도,n1풍향,n1풍속,n1강수량,n1해면기압,n1이슬점 온도,...,n6이슬점 온도,n7일시,n7평균기온,n7최고기온,n7습도,n7풍향,n7풍속,n7강수량,n7해면기압,n7이슬점 온도
0,1996-01-01,-3.639583,0.5000,-8.0875,48.419643,218.750000,1.169643,0.000000,1022.948810,-13.566369,...,-6.596667,3.833333,4.566667,3.133333,69.777778,123.333333,2.555556,0.000000,1019.444444,-2.877778
1,1996-01-02,0.900000,3.7500,-2.8125,71.203125,165.000000,2.515625,0.446631,1017.043750,-3.859375,...,-18.627143,6.112500,13.100000,-1.400000,44.841667,206.666667,3.625000,0.000000,1023.658333,-6.316667
2,1996-01-03,-6.095313,0.0750,-8.7750,52.091518,253.750000,4.433265,0.446631,1021.464062,-14.331250,...,-21.976429,1.954167,5.333333,-0.766667,53.358333,60.000000,3.000000,0.000000,1028.558333,-7.183333
3,1996-01-04,-7.325000,-1.9750,-12.4375,59.312500,198.348983,1.484375,0.445600,1025.557812,-14.362500,...,-12.009286,5.637500,10.966667,0.366667,64.408333,240.000000,2.726190,0.000000,1019.637500,-0.950000
4,1996-01-05,-1.073214,1.1750,-3.0500,80.558036,230.524917,1.966518,1.623677,1021.357143,-4.158482,...,-12.091071,6.191667,10.866667,2.066667,67.700000,210.000000,2.375000,0.000000,1021.525000,0.533333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9370,2021-08-27,23.270312,24.9500,21.7125,82.468750,111.250000,0.906250,0.880792,1011.759375,20.028125,...,16.523214,25.229167,28.333333,22.500000,78.830556,240.000000,2.125000,-0.047822,1012.870833,21.366667
9371,2021-08-28,24.135938,28.5375,20.8125,77.921875,115.000000,1.500000,0.000000,1013.007812,19.660937,...,16.480000,26.490476,30.100000,23.133333,81.301190,106.666667,1.404762,-0.739609,1012.662500,22.993452
9372,2021-08-29,23.090625,26.2000,20.8500,83.125000,162.500000,1.031250,1.669771,1013.484375,19.915625,...,17.045000,25.787500,29.033333,23.233333,87.795833,210.000000,2.083333,-0.702848,1013.058333,23.450000
9373,2021-08-30,23.782813,28.3375,20.4375,77.578125,183.750000,0.937500,0.000000,1015.429688,19.173437,...,17.566429,27.412500,30.700000,25.233333,83.704167,220.000000,1.916667,-0.625120,1012.554167,24.179167


In [16]:
df_near.columns

Index(['군집', '일시', '평균기온', '최고기온', '최저기온', '습도', '풍향', '풍속', '강수량', '해면기압',
       '이슬점 온도'],
      dtype='object')

In [17]:
# 주변국과 한파를 merge
cold_near = pd.merge(df_coldwave,tdf, how='right',on='일시')

In [18]:
cold_near['일시'].sort_values(ascending=False).value_counts()

2021-08-31    1
2004-07-09    1
2004-07-25    1
2004-07-24    1
2004-07-23    1
             ..
2013-02-08    1
2013-02-07    1
2013-02-06    1
2013-02-05    1
1996-01-01    1
Name: 일시, Length: 9375, dtype: int64

In [ ]:
cold_near

In [19]:
# 재난일어난날 컬럼을 바이너리로 만들기 
cold_near['재난일어난날'] = cold_near['재난일어난날'].fillna(0)

In [20]:
cold_near['재난일어난날'] =cold_near['재난일어난날'].apply(lambda x: x if str(x) == '0' else '1')

In [21]:
# 쓸모없는 column 제거
cold_near.drop('일시',axis=1,inplace=True)

In [22]:
# 전부 뉴메릭으로 만들기 
cold_near['재난일어난날']=cold_near['재난일어난날'].astype(int)

In [23]:
# 위치를 원핫인코딩하기
cold_near = pd.get_dummies(cold_near)

In [24]:
# 재난일어난날 2주뒤로 미뤄서 예측값 찾기 
cold_near['재난일어난날'] = cold_near['재난일어난날'].shift(periods=14, axis=0)

In [25]:
# 뒤로미룬만큼 생긴 nan값 제거 
cold_near.dropna(inplace=True)

In [26]:
cold_near

,재난일어난날,n1일시,n1평균기온,n1최고기온,n1습도,n1풍향,n1풍속,n1강수량,n1해면기압,n1이슬점 온도,...,n6이슬점 온도,n7일시,n7평균기온,n7최고기온,n7습도,n7풍향,n7풍속,n7강수량,n7해면기압,n7이슬점 온도
14,1.0,-0.675000,0.6125,-2.0000,83.601190,288.750000,1.927083,0.517108,1019.924479,-3.124926,...,-19.176429,0.337500,2.100000,-1.633333,78.000000,236.666667,3.583333,-0.058884,1028.150000,-3.254167
15,1.0,-6.378125,-3.1250,-8.9375,57.859375,299.274917,3.796875,0.000000,1023.650000,-13.739062,...,-21.121429,0.970833,2.833333,-1.600000,63.079167,43.333333,2.875000,0.000000,1028.450000,-5.525000
16,0.0,-7.145982,-3.2125,-10.1125,59.627232,243.750000,1.600446,0.000000,1027.639955,-14.147098,...,-19.753214,0.375000,1.500000,-1.333333,81.823333,33.333333,3.541667,-0.058884,1028.104167,-2.404167
17,1.0,-5.994420,-2.3000,-8.8625,55.169643,285.000000,1.718750,0.000000,1028.115625,-13.856920,...,-18.632143,0.041667,1.366667,-1.566667,68.062500,130.000000,2.500000,1.933333,1029.966667,-5.720833
18,1.0,-5.354985,-0.7000,-9.5000,64.029018,175.848983,0.908482,0.000000,1022.971652,-11.586384,...,-15.828214,-0.075000,2.900000,-3.566667,69.304167,233.333333,1.750000,0.000000,1024.954167,-5.658333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9370,0.0,23.270312,24.9500,21.7125,82.468750,111.250000,0.906250,0.880792,1011.759375,20.028125,...,16.523214,25.229167,28.333333,22.500000,78.830556,240.000000,2.125000,-0.047822,1012.870833,21.366667
9371,0.0,24.135938,28.5375,20.8125,77.921875,115.000000,1.500000,0.000000,1013.007812,19.660937,...,16.480000,26.490476,30.100000,23.133333,81.301190,106.666667,1.404762,-0.739609,1012.662500,22.993452
9372,0.0,23.090625,26.2000,20.8500,83.125000,162.500000,1.031250,1.669771,1013.484375,19.915625,...,17.045000,25.787500,29.033333,23.233333,87.795833,210.000000,2.083333,-0.702848,1013.058333,23.450000
9373,0.0,23.782813,28.3375,20.4375,77.578125,183.750000,0.937500,0.000000,1015.429688,19.173437,...,17.566429,27.412500,30.700000,25.233333,83.704167,220.000000,1.916667,-0.625120,1012.554167,24.179167


In [27]:
cold_near['재난일어난날'] = cold_near['재난일어난날'].astype(int)

In [28]:
cold_near

,재난일어난날,n1일시,n1평균기온,n1최고기온,n1습도,n1풍향,n1풍속,n1강수량,n1해면기압,n1이슬점 온도,...,n6이슬점 온도,n7일시,n7평균기온,n7최고기온,n7습도,n7풍향,n7풍속,n7강수량,n7해면기압,n7이슬점 온도
14,1,-0.675000,0.6125,-2.0000,83.601190,288.750000,1.927083,0.517108,1019.924479,-3.124926,...,-19.176429,0.337500,2.100000,-1.633333,78.000000,236.666667,3.583333,-0.058884,1028.150000,-3.254167
15,1,-6.378125,-3.1250,-8.9375,57.859375,299.274917,3.796875,0.000000,1023.650000,-13.739062,...,-21.121429,0.970833,2.833333,-1.600000,63.079167,43.333333,2.875000,0.000000,1028.450000,-5.525000
16,0,-7.145982,-3.2125,-10.1125,59.627232,243.750000,1.600446,0.000000,1027.639955,-14.147098,...,-19.753214,0.375000,1.500000,-1.333333,81.823333,33.333333,3.541667,-0.058884,1028.104167,-2.404167
17,1,-5.994420,-2.3000,-8.8625,55.169643,285.000000,1.718750,0.000000,1028.115625,-13.856920,...,-18.632143,0.041667,1.366667,-1.566667,68.062500,130.000000,2.500000,1.933333,1029.966667,-5.720833
18,1,-5.354985,-0.7000,-9.5000,64.029018,175.848983,0.908482,0.000000,1022.971652,-11.586384,...,-15.828214,-0.075000,2.900000,-3.566667,69.304167,233.333333,1.750000,0.000000,1024.954167,-5.658333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9370,0,23.270312,24.9500,21.7125,82.468750,111.250000,0.906250,0.880792,1011.759375,20.028125,...,16.523214,25.229167,28.333333,22.500000,78.830556,240.000000,2.125000,-0.047822,1012.870833,21.366667
9371,0,24.135938,28.5375,20.8125,77.921875,115.000000,1.500000,0.000000,1013.007812,19.660937,...,16.480000,26.490476,30.100000,23.133333,81.301190,106.666667,1.404762,-0.739609,1012.662500,22.993452
9372,0,23.090625,26.2000,20.8500,83.125000,162.500000,1.031250,1.669771,1013.484375,19.915625,...,17.045000,25.787500,29.033333,23.233333,87.795833,210.000000,2.083333,-0.702848,1013.058333,23.450000
9373,0,23.782813,28.3375,20.4375,77.578125,183.750000,0.937500,0.000000,1015.429688,19.173437,...,17.566429,27.412500,30.700000,25.233333,83.704167,220.000000,1.916667,-0.625120,1012.554167,24.179167


In [ ]:
# 한파주변국 데이터 전처리 끝 모델돌리기 

In [29]:
y = cold_near.재난일어난날
x = cold_near.drop('재난일어난날',axis=1)

In [30]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

In [31]:
models = [lr,dtc,rfc,lgbm]
# result = []
for i in models:
    i.fit(x_train,y_train)
    pred = i.predict(x_test)
    proba = i.predict_proba(x_test)[:,1]
    
    print(i.__class__.__name__)
    print(classification_report(y_test,pred))
    print('roc_auc_score :',roc_auc_score(y_test,proba))
    print('-'*50)

LogisticRegression
              precision    recall  f1-score   support

           0       0.91      0.96      0.94      1635
           1       0.57      0.37      0.45       238

    accuracy                           0.88      1873
   macro avg       0.74      0.66      0.69      1873
weighted avg       0.87      0.88      0.87      1873

roc_auc_score : 0.9262560069899518
--------------------------------------------------
DecisionTreeClassifier
              precision    recall  f1-score   support

           0       0.92      0.93      0.93      1635
           1       0.49      0.48      0.49       238

    accuracy                           0.87      1873
   macro avg       0.71      0.71      0.71      1873
weighted avg       0.87      0.87      0.87      1873

roc_auc_score : 0.7052052013465937
--------------------------------------------------
RandomForestClassifier
              precision    recall  f1-score   support

           0       0.91      0.97      0.94      1635


In [ ]:
#f1 score가 너무 낮게나왔으니 임밸런스 데이터를 제거하고 다시 

In [32]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [33]:
scaler.fit(x_train)

StandardScaler()

In [34]:
x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [35]:
y_train.value_counts()

0    6616
1     872
Name: 재난일어난날, dtype: int64

In [36]:
import imblearn
from imblearn.over_sampling import SMOTE

In [37]:
smote = SMOTE(random_state=0)
x_train_sm,y_train_sm = smote.fit_resample(x_train_scaled,y_train)

In [38]:
y_train_sm.value_counts()

0    6616
1    6616
Name: 재난일어난날, dtype: int64

In [41]:
models = [lr,dtc,rfc,lgbm]
# result = []
for i in models:
    i.fit(x_train_sm,y_train_sm)
    pred = i.predict(x_test_scaled)
    proba = i.predict_proba(x_test_scaled)[:,1]
    
    print(i.__class__.__name__)
    print(classification_report(y_test,pred))
    print('roc_auc_score :',roc_auc_score(y_test,proba))
    print('-'*50)
    

LogisticRegression
              precision    recall  f1-score   support

           0       0.98      0.83      0.90      1635
           1       0.44      0.91      0.59       238

    accuracy                           0.84      1873
   macro avg       0.71      0.87      0.75      1873
weighted avg       0.92      0.84      0.86      1873

0.5912806539509536
roc_auc_score : 0.9293655076709583
--------------------------------------------------
DecisionTreeClassifier
              precision    recall  f1-score   support

           0       0.93      0.89      0.91      1635
           1       0.40      0.51      0.45       238

    accuracy                           0.84      1873
   macro avg       0.66      0.70      0.68      1873
weighted avg       0.86      0.84      0.85      1873

0.4510166358595194
roc_auc_score : 0.7009508390512168
--------------------------------------------------
RandomForestClassifier
              precision    recall  f1-score   support

           0    

In [ ]:
# 랜덤포레스트 모델의 하이퍼파라미터를 랜덤서치로 최적의 값 찾기 .

In [40]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

In [42]:
est = RandomForestClassifier(n_jobs=-1, n_estimators=500)
rf_p_dist = {'max_depth':[3,5,10,None],
                'n_estimators':[100,200,300,400,500],
                'max_features':randint(1,3),
                'criterion':['gini','entropy'],
                'bootstrap':['True','False'],
                'min_samples_leaf':randint(1,4)}

def hypertuning_rscv(est,p_distr, nbr_iter,x,y):
    rdmserch = RandomizedSearchCV(est,param_distributions=p_distr,
                                 n_jobs=-1, n_iter=nbr_iter, cv=9)
    
    rdmserch.fit(x,y)
    ht_params = rdmserch.best_params_
    ht_score = rdmserch.best_score_
    return ht_params, ht_score

rf_parameters,rf_ht_score = hypertuning_rscv(est,rf_p_dist,40,x,y)

y_pred = rfc.predict(x_test)

# classifier=RandomForestClassifier

In [43]:
rf_parameters

{'bootstrap': 'False',
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 2,
 'min_samples_leaf': 3,
 'n_estimators': 100}

In [44]:
rf_ht_score

0.8911436652791119

In [45]:
rfc = RandomForestClassifier(n_jobs=-1,bootstrap=True,
 criterion= 'entropy',
 max_depth= 10,
 max_features= 1,
 min_samples_leaf= 2,
 n_estimators=100)

In [46]:
from sklearn.model_selection import cross_val_score
cross_val = cross_val_score(rfc,x,y,cv=10,scoring='accuracy').mean()

In [47]:
cross_val

0.8829177270612704

In [48]:
models = [lr,dtc,rfc,lgbm]
# result = []
for i in models:
    i.fit(x_train_sm,y_train_sm)
    pred = i.predict(x_test_scaled)
    proba = i.predict_proba(x_test_scaled)[:,1]
    
    print(i.__class__.__name__)
    print(classification_report(y_test,pred))
    print('roc_auc_score :',roc_auc_score(y_test,proba))
    print('-'*50)
    

LogisticRegression
              precision    recall  f1-score   support

           0       0.98      0.83      0.90      1635
           1       0.44      0.91      0.59       238

    accuracy                           0.84      1873
   macro avg       0.71      0.87      0.75      1873
weighted avg       0.92      0.84      0.86      1873

roc_auc_score : 0.9293655076709583
--------------------------------------------------
DecisionTreeClassifier
              precision    recall  f1-score   support

           0       0.93      0.89      0.91      1635
           1       0.40      0.50      0.45       238

    accuracy                           0.84      1873
   macro avg       0.66      0.70      0.68      1873
weighted avg       0.86      0.84      0.85      1873

roc_auc_score : 0.6979723999691619
--------------------------------------------------
RandomForestClassifier
              precision    recall  f1-score   support

           0       0.99      0.80      0.89      1635
